In [1]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.utils import plot_model
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import model_from_json


import cv2
import csv
import re
from PIL import Image
from PIL import ImageEnhance
from PIL import ImageStat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image as imge
from keras.applications.resnet50 import preprocess_input, decode_predictions
import math



/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [3]:
FilePath = "/home/nramvinojen/Programs/Workbench/24Jan2019/"
NewFVlen = 48
Class = 8
#RunFolder = "Default"
RunFolder = "28Jan2019"

In [4]:
# load json and create model
json_file = open(FilePath + "Dataset/ModelAndWeights/FVReduced_2048to" + str(NewFVlen) + ".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(FilePath + "Dataset/ModelAndWeights/FVReduced_2048to" + str(NewFVlen) + ".h5")
print("Loaded model from disk")
loaded_model.summary()

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
A1 (InputLayer)              (None, 2048)              0         
_________________________________________________________________
A2 (Dense)                   (None, 92)                188508    
_________________________________________________________________
A3 (Dropout)                 (None, 92)                0         
_________________________________________________________________
A4 (Dense)                   (None, 48)                4464      
_________________________________________________________________
A5 (Dropout)                 (None, 48)                0         
Total params: 192,972
Trainable params: 192,972
Non-trainable params: 0
_________________________________________________________________


In [7]:
ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
data = list(csv.reader(ClassInfo))
NumClass = int(data[0][0]) #No of class
count = 0 #USed in writing frames to text file

In [8]:
for k in range(NumClass): #Looping through all class
    ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
    data = list(csv.reader(ClassInfo))
    NumTrainDataSet = int(data[k+1][1])  #No of Training datasets
   
    #Trainingdata   
    for i in range(NumTrainDataSet): #Looping through all datasets in a class  
        FixationInfo = pd.read_csv(FilePath +"Dataset/Dataset_Preprocessed/Fixations/C"+str(k)+"_TR_"+str(i)+"_fixations.csv", usecols=[0])
        data = FixationInfo.values
        fixation_data = [item for sublist in data for item in sublist]
        NumFixation = len(fixation_data)-1 #no of fixations in a data set
        
        # load dataset
        dataframe = pd.read_csv(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_Original/C" + str(k) + "_TR_" + str(i) + '_Fixation_FeatureVector.csv', header=None)
        dataset = dataframe.values
        X = dataset[:,0:2048].astype(float)
           
        TempFeatureVector = loaded_model.predict(X)
                 
        #Fixation duration total
        FixationDuration = pd.read_csv(FilePath +"Dataset/Dataset_Preprocessed/Fixations/C"+str(k)+"_TR_"+ str(i) +"_fixations.csv", usecols=[2])
        data = FixationDuration.values
        NormFixationDurationList = [item for sublist in data for item in sublist]

        TotalDuration = sum(NormFixationDurationList) - NormFixationDurationList[NumFixation]
        NormFixationDurationList[:] = [m / TotalDuration for m in NormFixationDurationList]   
        
        #Computing Feature Vector for Fixation Data       
        for j in range(NumFixation):#Lopping through all fixations
            FeatureVector = list(TempFeatureVector[j])
           # FeatureVector = [item for sublist in TempFeatureVector[j] for item in sublist] #To convert nd array into list
            #multiply FeatureVector with normalized fixation duration weight
            FeatureVector = [y * NormFixationDurationList[j] for y in FeatureVector]
             
            #Writing Feature Vector of each data set in a separate file, no of rows = no of fixations    
            with open(FilePath + "Runs/" + RunFolder + "/FV" + str(NewFVlen) + "_fromResnet/Fixation_WeightNormalized/C" + str(k) + "_TR_" + str(i) + '_Fixation_FeatureVector.csv', 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(FeatureVector) 
                
        #Computing Combined Feature Vector
        FixationFeatureFile = open(FilePath + "Runs/" + RunFolder + "/FV" + str(NewFVlen) + "_fromResnet/Fixation_WeightNormalized/C" + str(k)+"_TR_" + str(i) + '_Fixation_FeatureVector.csv',"r")
        FeaturePoint = list(csv.reader(FixationFeatureFile))
        CombinedFeatureVector = [0]*len(FeaturePoint[0])
        for m in range(len(FeaturePoint[0])):
            for n in range(NumFixation):
                CombinedFeatureVector[m] += float(FeaturePoint[n][m])
                
        #Writing combined Feature Vector of each data set in a file, no of rows = no of data sets          
        with open(FilePath + "Runs/" + RunFolder + "/FV" + str(NewFVlen) + "_fromResnet/Fixation_WeightedSum_Combined_Csv/TR_CombinedFixation_FeatureVector.csv", 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(CombinedFeatureVector) 


In [9]:
for k in range(NumClass): #Looping through all class
    ClassInfo = open(FilePath + "Dataset/Dataset_Preprocessed/ClassInfo.csv","r")
    data = list(csv.reader(ClassInfo))
    NumTestDataSet = int(data[k+1][2])  #No of Training datasets
   
    #TestingData   
    for i in range(NumTestDataSet): #Looping through all datasets in a class  
        FixationInfo = pd.read_csv(FilePath +"Dataset/Dataset_Preprocessed/Fixations/C"+str(k)+"_Test_"+str(i)+"_fixations.csv", usecols=[0])
        data = FixationInfo.values
        fixation_data = [item for sublist in data for item in sublist]
        NumFixation = len(fixation_data)-1 #no of fixations in a data set
        
        # load dataset
        dataframe = pd.read_csv(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Fixation_Original/C" + str(k) + "_Test_" + str(i) + '_Fixation_FeatureVector.csv', header=None)
        dataset = dataframe.values
        X = dataset[:,0:2048].astype(float)
           
        TempFeatureVector = loaded_model.predict(X)
                 
        #Fixation duration total
        FixationDuration = pd.read_csv(FilePath +"Dataset/Dataset_Preprocessed/Fixations/C"+str(k)+"_Test_"+ str(i) +"_fixations.csv", usecols=[2])
        data = FixationDuration.values
        NormFixationDurationList = [item for sublist in data for item in sublist]

        TotalDuration = sum(NormFixationDurationList) - NormFixationDurationList[NumFixation]
        NormFixationDurationList[:] = [m / TotalDuration for m in NormFixationDurationList]   
        
        #Computing Feature Vector for Fixation Data       
        for j in range(NumFixation):#Lopping through all fixations
            FeatureVector = list(TempFeatureVector[j])
           # FeatureVector = [item for sublist in TempFeatureVector[j] for item in sublist] #To convert nd array into list
            #multiply FeatureVector with normalized fixation duration weight
            FeatureVector = [y * NormFixationDurationList[j] for y in FeatureVector]
             
            #Writing Feature Vector of each data set in a separate file, no of rows = no of fixations    
            with open(FilePath + "Runs/" + RunFolder + "/FV" + str(NewFVlen) + "_fromResnet/Fixation_WeightNormalized/C" + str(k) + "_Test_" + str(i) + '_Fixation_FeatureVector.csv', 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(FeatureVector) 
                
        #Computing Combined Feature Vector
        FixationFeatureFile = open(FilePath + "Runs/" + RunFolder + "/FV" + str(NewFVlen) + "_fromResnet/Fixation_WeightNormalized/C" + str(k)+"_Test_" + str(i) + '_Fixation_FeatureVector.csv',"r")
        FeaturePoint = list(csv.reader(FixationFeatureFile))
        CombinedFeatureVector = [0]*len(FeaturePoint[0])
        for m in range(len(FeaturePoint[0])):
            for n in range(NumFixation):
                CombinedFeatureVector[m] += float(FeaturePoint[n][m])
                
        #Writing combined Feature Vector of each data set in a file, no of rows = no of data sets          
        with open(FilePath + "Runs/" + RunFolder + "/FV" + str(NewFVlen) + "_fromResnet/Fixation_WeightedSum_Combined_Csv/Test_CombinedFixation_FeatureVector.csv", 'a', newline='') as graphFile:     
                graphFileWriter = csv.writer(graphFile, delimiter = ',')
                graphFileWriter.writerow(CombinedFeatureVector) 

In [12]:
#target Image

# Training load dataset
dataframe = pd.read_csv(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Target_Combined_Csv/TR_CombinedTarget_FeatureVector.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:2048].astype(float)

TempFeatureVector = loaded_model.predict(X)

for h in range(len(X)) :
    X_Temp = TempFeatureVector[h]       
    with open(FilePath + "Runs/" + RunFolder + "/FV" + str(NewFVlen) + "_fromResnet/Target_Combined_Csv/TR_CombinedTarget_FeatureVector.csv", 'a', newline='') as graphFile:     
            graphFileWriter = csv.writer(graphFile, delimiter = ',')
            graphFileWriter.writerow(X_Temp) 
            
            


In [13]:
# Testing load dataset
dataframe = pd.read_csv(FilePath + "Runs/" + RunFolder + "/FV2048_Resnet/Target_Combined_Csv/Test_CombinedTarget_FeatureVector.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:2048].astype(float)

TempFeatureVector = loaded_model.predict(X)

for h in range(len(X)) :
    X_Temp = TempFeatureVector[h]       
    with open(FilePath + "Runs/" + RunFolder + "/FV" + str(NewFVlen) + "_fromResnet/Target_Combined_Csv/Test_CombinedTarget_FeatureVector.csv", 'a', newline='') as graphFile:     
            graphFileWriter = csv.writer(graphFile, delimiter = ',')
            graphFileWriter.writerow(X_Temp) 